In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://www.betexplorer.com/basketball/usa/nba/fixtures/"
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')


In [11]:
table_matches = soup.find('table', attrs={'class':"table-main table-main--leaguefixtures h-mb15 js-tablebanner-t js-tablebanner-ntb"})
data = []
rows = table_matches.find_all('tr')
for row in rows:
    utils = []
    cols = row.find_all('td')
    
    
    for element in cols:
        utils.append(element.text)
    data.append(utils)

df = pd.DataFrame(data,columns=["Date","Match", "B","C","1","2"])

In [31]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Specify the path to chromedriver
driver_path = '/Users/jamiewelsh/chromedriver_mac64'  # Replace with the actual path to chromedriver

# Set up the Selenium WebDriver
driver = webdriver.Chrome()

# Open the webpage using Selenium
driver.get(URL)

# Wait for a few seconds to ensure that dynamic content is loaded
time.sleep(5)  # You may need to adjust this depending on the page load time

# Get the updated HTML content
html = driver.page_source

# Close the WebDriver
driver.quit()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the table containing the matches
table_matches = soup.find('table', attrs={'class': "table-main table-main--leaguefixtures h-mb15 js-tablebanner-t js-tablebanner-ntb"})

data = []
rows = table_matches.find_all('tr')

for row in rows:
    utils = []
    cols = row.find_all('td')

    for element in cols:
        utils.append(element.text)
        
    data.append(utils)

# Create a DataFrame with the scraped data
df = pd.DataFrame(data, columns=["Date", "Match", "B", "C", "1", "2"])

df.drop(columns=['B', 'C'], axis=1, inplace=True)
df.dropna(subset={'1'}, inplace=True)
df = df[df['1'] != '\xa0']
df.reset_index(inplace=True)
for index, row in df.iterrows():
    if row['Date'] == '\xa0' and index > 0:
        df.at[index, 'Date'] = df.at[index - 1, 'Date']

df.head()


,index,Date,Match,1,2
0,1,Tomorrow 01:00,Cleveland Cavaliers - Washington Wizards,1.22,4.45
1,2,Tomorrow 01:00,Indiana Pacers - Milwaukee Bucks,2.35,1.62
2,3,Tomorrow 01:30,Atlanta Hawks - Oklahoma City Thunder,2.03,1.81
3,4,Tomorrow 02:00,Houston Rockets - Brooklyn Nets,1.38,3.10
4,5,Tomorrow 02:00,Memphis Grizzlies - Toronto Raptors,1.87,1.97


In [35]:
df['1'] = df['1'].astype(float)
df['2'] = df['2'].astype(float)


df['Implied_Prob_1'] = 1/df['1']
df['Implied_Prob_2'] = 1/df['2']

In [37]:
df['Fair_Prob_1'] = df['Implied_Prob_1']/(df['Implied_Prob_1'] + df['Implied_Prob_2'])
df['Fair_Prob_2'] = df['Implied_Prob_2']/(df['Implied_Prob_1'] + df['Implied_Prob_2'])

In [38]:
df

,index,Date,Match,1,2,Implied_Prob_1,Implied_Prob_2,Fair_Prob_1,Fair_Prob_2
0,1,Tomorrow 01:00,Cleveland Cavaliers - Washington Wizards,1.22,4.45,0.819672,0.224719,0.784832,0.215168
1,2,Tomorrow 01:00,Indiana Pacers - Milwaukee Bucks,2.35,1.62,0.425532,0.617284,0.408060,0.591940
2,3,Tomorrow 01:30,Atlanta Hawks - Oklahoma City Thunder,2.03,1.81,0.492611,0.552486,0.471354,0.528646
3,4,Tomorrow 02:00,Houston Rockets - Brooklyn Nets,1.38,3.10,0.724638,0.322581,0.691964,0.308036
4,5,Tomorrow 02:00,Memphis Grizzlies - Toronto Raptors,1.87,1.97,0.534759,0.507614,0.513021,0.486979
5,6,Tomorrow 02:00,Minnesota Timberwolves - New Orleans Pelicans,1.37,3.20,0.729927,0.312500,0.700219,0.299781
6,7,Tomorrow 02:30,Dallas Mavericks - Portland Trail Blazers,1.15,5.70,0.869565,0.175439,0.832117,0.167883
7,8,Tomorrow 02:30,New York Knicks - Chicago Bulls,1.24,4.11,0.806452,0.243309,0.768224,0.231776
8,9,Tomorrow 03:00,Phoenix Suns - Los Angeles Clippers,2.31,1.64,0.432900,0.609756,0.415190,0.584810
9,10,Tomorrow 03:00,Utah Jazz - Detroit Pistons,1.26,3.94,0.793651,0.253807,0.757692,0.242308
